In [4]:
import sys
sys.path.append("/home/elliottqian/Documents/PycharmProjects/deeplearning_notebook/mxnet_study")
from movielens.fm_basic import MyMutEmbedding
from movielens.preprocess import get_csv_file_batch_tier

import mxnet as mx
from mxnet import gluon
from mxnet import autograd
from sklearn import metrics

try:
    my_ctx = mx.gpu()
except:
    my_ctx = mx.cpu()

/mnt/D/Ubuntu/package/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [5]:
csv_path = '/home/elliottqian/Documents/PycharmProjects/deeplearning_notebook/mxnet_study/movielens/part.csv'
dim_array = [2, 7, 21, 3706, 301]

In [6]:
from movielens.simple_fm_for_movielens import simple_fm_net
from movielens.simple_fm_for_movielens import get_trainer

gpu(0)


In [7]:
fm_net = simple_fm_net(dim_array, 10, my_ctx)

In [8]:
fm_net.collect_params()

sequential0_ (
  Parameter sequential0_mymutembedding0_embedding0_weight (shape=(2, 10), dtype=<class 'numpy.float32'>)
  Parameter sequential0_mymutembedding0_embedding1_weight (shape=(7, 10), dtype=<class 'numpy.float32'>)
  Parameter sequential0_mymutembedding0_embedding2_weight (shape=(21, 10), dtype=<class 'numpy.float32'>)
  Parameter sequential0_mymutembedding0_embedding3_weight (shape=(3706, 10), dtype=<class 'numpy.float32'>)
  Parameter sequential0_mymutembedding0_embedding4_weight (shape=(301, 10), dtype=<class 'numpy.float32'>)
  Parameter sequential0_dense0_weight (shape=(10, 0), dtype=<class 'numpy.float32'>)
  Parameter sequential0_dense0_bias (shape=(10,), dtype=<class 'numpy.float32'>)
  Parameter sequential0_dense1_weight (shape=(1, 0), dtype=<class 'numpy.float32'>)
  Parameter sequential0_dense1_bias (shape=(1,), dtype=<class 'numpy.float32'>)
)

In [9]:
trainer = get_trainer(fm_net, sgd_type="sgd", learning_rate=0.5)

In [18]:
import sys
sys.path.append("/home/elliottqian/Documents/PycharmProjects/deeplearning_notebook/mxnet_study")
from movielens.fm_basic import MyMutEmbeddinga

In [19]:
from movielens.preprocess import get_csv_file_batch_tier

In [20]:
data_csv='/home/elliottqian/Documents/PycharmProjects/deeplearning_notebook/mxnet_study/movielens/part.csv'
data_iter = get_csv_file_batch_tier(data_csv, data_shape=(6), batch=2)  # debug_skip_load(self)
data_iter.debug_skip_load()

In [21]:
data_iter.next().data[0]


[[  1.00000000e+00   1.00000000e+00   2.00000000e+00   1.70000000e+01
    1.09000000e+03   1.30000000e+01]
 [  1.00000000e+00   1.00000000e+00   1.00000000e+00   1.90000000e+01
    1.09000000e+03   1.30000000e+01]]
<NDArray 2x6 @cpu(0)>

In [22]:
text_X = data_iter.next().data[0][:, 1:]
text_y = data_iter.next().data[0][:, 0]

In [8]:
dim_array = [2, 7, 21, 3706, 301]

In [9]:
class InnerProductLayer(gluon.nn.Block):
    
    def __init__(self, **kwargs):
        super(InnerProductLayer, self).__init__(**kwargs)
        
    def forward(self, x):
        """
        
        :param x: 
        :return: 
        """
        y = mx.nd.swapaxes(x, 1, 2)
        result = mx.nd.batch_dot(x, y)
        result = mx.nd.flatten(result)
        return result

In [10]:
my_net = gluon.nn.Sequential()
with my_net.name_scope():
    my_net.add(MyMutEmbedding(dim_array, 8, True))
    my_net.add(InnerProductLayer())
    my_net.add(gluon.nn.Dense(1, activation='sigmoid'))

In [11]:
my_net.collect_params()

sequential0_ (
  Parameter sequential0_mymutembedding0_embedding0_weight (shape=(2, 8), dtype=<class 'numpy.float32'>)
  Parameter sequential0_mymutembedding0_embedding1_weight (shape=(7, 8), dtype=<class 'numpy.float32'>)
  Parameter sequential0_mymutembedding0_embedding2_weight (shape=(21, 8), dtype=<class 'numpy.float32'>)
  Parameter sequential0_mymutembedding0_embedding3_weight (shape=(3706, 8), dtype=<class 'numpy.float32'>)
  Parameter sequential0_mymutembedding0_embedding4_weight (shape=(301, 8), dtype=<class 'numpy.float32'>)
  Parameter sequential0_dense0_weight (shape=(1, 0), dtype=<class 'numpy.float32'>)
  Parameter sequential0_dense0_bias (shape=(1,), dtype=<class 'numpy.float32'>)
)

In [12]:
loss = gluon.loss.SigmoidBinaryCrossEntropyLoss(from_sigmoid=True)
my_net.initialize()

In [13]:
my_net(data_iter.next().data[0][:, 1:])


[[ 0.50034362]
 [ 0.50038183]]
<NDArray 2x1 @cpu(0)>

In [14]:
trainer = gluon.Trainer(my_net.collect_params(), 'AdaGrad', {'learning_rate': 0.3})

In [15]:
data_iter = get_csv_file_batch_tier(data_csv, data_shape=(6), batch=10)
data_iter.debug_skip_load()
i = -1
for _ in range(2001):
    i += 1
    data_label = data_iter.next().data[0]
    train_X = data_label[:, 1:].as_in_context(mx.cpu())
    train_y = data_label[:, 0].as_in_context(mx.cpu())
    with autograd.record():
        output = my_net(train_X)
        loss_num = loss(train_y, output) 
        if i % 200 == 0:
            print(mx.ndarray.sum(loss_num) / 10)
    loss_num.backward()
    trainer.step(10)


[ 13.80959034]
<NDArray 1 @cpu(0)>



[ 2.76323366]
<NDArray 1 @cpu(0)>



[ 2.76315641]
<NDArray 1 @cpu(0)>



[ 2.76313519]
<NDArray 1 @cpu(0)>



[ 2.76312613]
<NDArray 1 @cpu(0)>


...


[ 2.76311755]
<NDArray 1 @cpu(0)>



[ 2.76311517]
<NDArray 1 @cpu(0)>



[ 2.7631135]
<NDArray 1 @cpu(0)>



[ 2.76311183]
<NDArray 1 @cpu(0)>



[ 2.76311064]
<NDArray 1 @cpu(0)>


In [16]:
my_net(data_iter.next().data[0][0:10,1:])


[[  9.99999881e-01]
 [  9.99999881e-01]
 [  1.70115470e-06]
 [  1.00000000e+00]
 [  9.99999762e-01]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  9.99999046e-01]
 [  1.00000000e+00]
 [  1.00000000e+00]]
<NDArray 10x1 @cpu(0)>

In [50]:
data_iter.next().data[0][0:10,0]


[ 1.  1.  0.  1.  1.  1.  1.  1.  1.  0.]
<NDArray 10 @cpu(0)>

In [24]:
from sklearn import cross_validation,metrics

/mnt/D/Ubuntu/package/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [29]:
test_y = [1, 0, 1, 1] 
prodict_prob_y = [0.8, 0.8, 0.9, 0.9]
test_auc = metrics.roc_auc_score(test_y,prodict_prob_y)

In [30]:
test_auc

0.83333333333333326